In [19]:
import pandas as pd
import os 
import glob
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import datetime 

# path of the folder
folder_path = '/Users/jud-05/Desktop/High Court/4Q_High Court'
new_path = '/Users/jud-05/Desktop/High Court/combined_4Q'
path = '/Users/jud-05/Desktop/High Court/combined_4Q/dcrt_4Q'

# Ensure the new directory exists
os.makedirs(new_path, exist_ok=True)

# reading all the excel files
file_names = glob.glob(folder_path + "/*.xls")

file_names

#create a file name without the directory and the extension and name each as the first word of the file
file_names2 = [os.path.basename(f) for f in file_names]
file_names2 = [f.split('.')[0] for f in file_names2]
file_names2 = [f.split()[0] for f in file_names2]
file_names2


#save files names as csv files and rename each file with the name of the original file
for f in file_names:
    df = pd.read_excel(f, skiprows=4)  # Skip the first 4 rows
    # Extract the base name of the file and save it as a .csv in the new directory
    base_name = os.path.basename(f)
    df.to_csv(f'{new_path}/{base_name}.csv', index=False)
    

KeyboardInterrupt: 

In [3]:
# Create a file name without the directory and the extension, appending a unique identifier
file_names2 = [os.path.basename(f).split('.')[0].split()[0] + '_' + str(i) for i, f in enumerate(file_names)]

# Save files as csv files
for f, base_name in zip(file_names, file_names2):
    df = pd.read_excel(f, skiprows=4)  # Skip the first 4 rows
    # Save it as a .csv in the new directory
    df.to_csv(f'{path}/{base_name}.csv', index=False)

In [5]:
# Get all the CSV files in the directory
all_files = glob.glob(path + "/*.csv")

all_files

# Combine all the CSV files into a single DataFrame and insert a column with the name of the file
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file_name'] = os.path.basename(filename)
    li.append(df)
       
combined_csv = pd.concat(li, axis=0, ignore_index=True)


#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_18269/2607724255.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_csv = pd.concat(li, axis=0, ignore_index=True)


In [6]:
combined_csv['file_name'] = combined_csv['file_name'].str.replace('.csv', '')

#remove in file_name the letters after the underscore
combined_csv['file_name'] = combined_csv['file_name'].str.split('_').str[0]
 #insert 

#move the file_name column to the first column
cols = combined_csv.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_csv = combined_csv[cols]

#drop Unnamed: 0 column
combined_csv = combined_csv.drop('Unnamed: 0', axis=1)

 #save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

In [7]:
#rename columns
# Simplified version of data.rename() function
combined_csv.rename(columns={
    'file_name': 'court_name',
    'Day': 'date_dd',
    'Month': 'date_mon',
    'Year': 'date_yyyy',
    'Code': 'caseid_type',
    'No.': 'caseid_no',
    'Day.1': 'filed_dd',
    'Month.1': 'filed_mon',
    'Year.1': 'filed_yyyy',
    'Name of Court': 'origcourt',
    'Code.1': 'appeal_type',
    'No..1': 'appeal_no',
    'Year.2': 'appeal_yyyy',
    '\n(Select)': 'case_type',
    'Name of Judge 1 or Magistrate/DR or Kadhi': 'bench_1',
    'Judge 2': 'bench_2',
    'Judge 3': 'bench_3',
    'Judge 4': 'bench_4',
    'Judge 5': 'bench_5',
    'Judge 6': 'bench_6',
    'Judge 7': 'bench_7',
    '(Select option)': 'comingfor',
    '(Select option).1': 'outcome',
    '(Select option).2': 'reason_adj',
    'Day.2': 'next_dd',
    'Month.2': 'next_mon',
    'Year.3': 'next_yyyy',
    'M': 'firstparty_m',
    'F': 'firstparty_f',
    'Org.': 'firstparty_org',
    'M.1': 'secondparty_m',
    'F.1': 'secondparty_f',
    'Org..1': 'secondparty_o',
    '(Y/N)': 'legalrep',
    'P \n(Enter No.)': 'p_witness',
    'D\n(Enter No.)': 'd_witness',
    '(Enter No.)': 'custody',
    '(Free text)': 'other_details'
}, inplace=True)

#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

In [6]:
#

In [8]:
def convert_to_datetime(combined_csv, day_col, month_col, year_col, new_col, index):
    date_str = combined_csv[day_col].astype(str) + "-" + combined_csv[month_col] + "-" + combined_csv[year_col].astype(str)
    combined_csv[new_col] = pd.to_datetime(date_str, format="%d-%b-%Y", errors="coerce")
    #data.insert(index, new_col, data.pop(new_col))
    return combined_csv

combined_csv = convert_to_datetime(combined_csv, "date_dd", "date_mon", "date_yyyy", "activity_date", 2)
combined_csv = convert_to_datetime(combined_csv, "filed_dd", "filed_mon", "filed_yyyy", "filed_date", 3)
for i in combined_csv['next_dd'].unique():
    if i == " ":
        combined_csv.loc[combined_csv[['next_dd', 'next_mon', 'next_yyyy']] == i, 'next_dd', 'next_mon', 'next_yyyy'] = ""
    else:
        combined_csv = convert_to_datetime(combined_csv, "next_dd", "next_mon", "next_yyyy", "next_date", 4)
        
combined_csv['activity_date'].value_counts()

activity_date
2024-04-09    2466
2024-04-08    2273
2024-04-11    2186
2024-04-04    2183
2024-04-15    1665
              ... 
2024-04-06      25
2024-05-01       5
2024-04-07       4
2024-06-02       1
2024-06-09       1
Name: count, Length: 69, dtype: int64

In [9]:
#create unique case id
combined_csv["case_identifier"] = combined_csv ['court_name'] + "-" + combined_csv["caseid_type"]+ "-" + combined_csv["caseid_no"].astype(str) + "/" + combined_csv["filed_date"].astype(str)
combined_csv.insert(1, "case_identifier", combined_csv.pop("case_identifier"))

combined_csv["case_identifier"].value_counts()

case_identifier
Mombasa-HCCRC-E017/2023-08-29          11
Homabay-HCCRC-E002/2024-01-17           7
Bomet-HCCRC-30/2018-11-23               6
Milimani-HCCHRPET-E181/2024-04-08       6
Kakamega-HCCCMISC-E014/2022-02-04       6
                                       ..
Milimani-HCCOMMMISC-E036/2024-01-18     1
Milimani-HCCOMMMISC-E038/2024-01-19     1
Milimani-HCCOMMMISC-E040/2024-01-19     1
Milimani-HCCOMMMISC-E050/2024-01-24     1
Maralal-HCCRREV-E001/2024-02-15         1
Name: count, Length: 46079, dtype: int64

In [10]:
#check duplicates
combined_csv.duplicated().sum()

#drop duplicates
combined_csv = combined_csv.drop_duplicates()



In [11]:
combined_csv['court_name'].value_counts()

court_name
Milimani      18229
Mombasa        2355
Kiambu         2273
Nakuru         1939
Machakos       1792
Eldoret        1650
Kakamega       1606
Meru           1566
Bungoma        1394
Muranga        1363
Kisumu         1224
Nyeri          1153
Kerugoya       1014
Malindi         990
Makueni         945
Thika           886
Kitale          797
Kajiado         774
Kitui           663
Kibera          628
Kisii           621
Naivasha        615
Kericho         613
Kapsabet        598
Embu            541
Voi             538
Nanyuki         521
Nyamira         494
Bomet           482
Nyandarua       481
Siaya           469
Busia           467
Migori          453
Vihiga          423
Narok           374
Homabay         343
Nyahururu       340
Chuka           325
Lodwar          272
Garissa         253
Kabarnet        244
Kilgoris        243
Garsen          225
Kwale           169
Kapenguria      146
Iten            137
Eldama           90
Marsabit         77
Maralal          31
Mandera  

In [12]:
combined_csv['case_type'].value_counts()

case_type
Civil Appeal                                                     8899
Family P&A Intestate                                             6782
Murder Case                                                      5387
Civil Case Miscellaneous                                         4619
Criminal Appeal                                                  4048
Commercial Matters                                               3266
Criminal Revision                                                2919
Civil Suit                                                       2494
Constitution and Human Rights Petition (Civil)                   1848
Criminal Miscellaneous Application                               1825
Commercial Miscellaneous                                         1377
Family P&A Testate                                               1094
Family P&A Citation                                              1089
Family P&A Ad Litem                                               985
Family Mis

In [13]:
def create_case_nature(combined_csv, old_col, new_col, index):
    case_nature = combined_csv[old_col].astype(str).apply(lambda x: 
        "CRIMINAL" if any(y in x for y in ["CR"]) 
        else "CIVIL")
    combined_csv[new_col] = case_nature
   #combined_csv.insert(index, new_col, combined_csv.pop(new_col))
    return combined_csv
combined_csv = create_case_nature(combined_csv, "caseid_type", "case_nature", 5)




In [14]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)     2753
Case Registered/Filed           2011
Directions given                1845
Hearing date set (in court)      748
Adjournment                      623
                                ... 
Ruling Date Set                    1
Mention date at the Registry       1
Oral submissions made              1
Limited Grant Issued               1
Grant revoked                      1
Name: count, Length: 73, dtype: int64

In [15]:
#generate function to generate filed cases width activity date before filed date
def filed_cases(combined_csv):
    filed = []
    for i, row in combined_csv.iterrows():
        if "Case Registered/Filed" in str(row['outcome']) and row['activity_date'] >= row['filed_date']:
            filed.append(1)
        else:
            filed.append(0)
    combined_csv['filed'] = filed
    return combined_csv

combined_csv = filed_cases(combined_csv)

combined_csv['filed'].value_counts()

filed
0    51866
1     1978
Name: count, dtype: int64

In [17]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)     2753
Case Registered/Filed           2011
Directions given                1845
Hearing date set (in court)      748
Adjournment                      623
                                ... 
Ruling Date Set                    1
Mention date at the Registry       1
Oral submissions made              1
Limited Grant Issued               1
Grant revoked                      1
Name: count, Length: 73, dtype: int64

In [18]:
#change outcome column to upper case
combined_csv['outcome'] = combined_csv['outcome'].str.upper()
def resolved_cases(combined_csv):
    resolved = []
    for i in combined_csv['outcome'].astype(str):
        if ("ABATED" in i or "CASE CLOSED" in i or "STRUCK OUT" in i or  "TERMINATED" in i or "DISMISS" in i or "GRANT CONFIRMED" in i or "GRANT REVOKED" in i or "JUDGMENT DELIVERED" in i or "ACQUITTAL" in i or "CONVICTED" in i or "LIMITED GRANT ISSUED" in i or "WITHDRAW" in i or "SETTLEMEN" in i or "REVISION DECLINED" in i or "RULING DELIVERED- ACCUSED DISC" in i or "RULING DELIVERED- CASE CLOSED" in i) and ("ORDER ISSUED - CASE CLOSED" not in i):
            resolved.append(1)
    
        else:
            resolved.append(0)
    combined_csv['resolved'] = resolved
    return combined_csv

combined_csv = resolved_cases(combined_csv)

total_resolved = combined_csv['resolved'].sum()
combined_csv[['outcome', 'resolved']] [combined_csv['resolved'] == 1].value_counts()
total_resolved






1055

In [17]:
#resolved by court name
resolved_court = combined_csv.groupby(['court_name'])['resolved'].sum().reset_index()
resolved_court['resolved'] = resolved_court['resolved'].astype(int)

resolved_court = resolved_court.sort_values(by=['court_name'], ascending=True)

resolved_data = combined_csv[combined_csv['resolved'] == 1]

resolved_data['court_name'].value_counts()

court_name
Milimani      12183
Mombasa        1947
Eldoret        1608
Nakuru         1601
Kiambu         1414
Kakamega       1232
Kisumu         1121
Kisii           952
Meru            922
Muranga         789
Machakos        643
Bungoma         610
Kitale          599
Embu            578
Naivasha        574
Malindi         570
Nyeri           526
Voi             520
Kerugoya        473
Kericho         467
Nanyuki         381
Homabay         377
Migori          369
Kitui           336
Kajiado         334
Makueni         314
Kabarnet        289
Nyahururu       278
Busia           277
Bomet           253
Nyamira         249
Garissa         247
Kibera          242
Vihiga          226
Lodwar          191
Thika           189
Kapsabet        187
Siaya           179
Chuka           179
Garsen          158
Narok           156
Eldama           90
Kwale            81
Kilgoris         78
Kapenguria       71
Marsabit         58
Iten             56
Nyandarua        28
Mandera          24
Maralal  

In [18]:
# generate age of resolved cases

def time_to_resolve(resolved_data):
    time_to_resolve = []
    for i in resolved_data['court_name']:
        time_to_resolve.append(i)
    resolved_data.loc[:, 'time_to_resolve'] = resolved_data['activity_date'] - resolved_data['filed_date']
    return resolved_data

resolved_data = time_to_resolve(resolved_data)

resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].dt.days
resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].fillna(0).astype(int)

#drop if time to resolve is less than 0
resolved_data = resolved_data[resolved_data['time_to_resolve'] >= 0]

resolved_data['time_to_resolve'].value_counts()

/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_12213/2198342833.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_data.loc[:, 'time_to_resolve'] = resolved_data['activity_date'] - resolved_data['filed_date']
/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_12213/2198342833.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].dt.days
/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_12213/2198342833.

time_to_resolve
0        536
1        475
6        226
2        182
7        169
        ... 
4173       1
10665      1
10995      1
4370       1
8452       1
Name: count, Length: 5934, dtype: int64

In [19]:
#combine  'bench_1', 'bench_2', 'bench_3', 'bench_4', 'bench_5', 'bench_6','bench_7 as a single column
resolved_data['bench'] = resolved_data[['bench_1', 'bench_2', 'bench_3', 'bench_4', 'bench_5', 'bench_6', 'bench_7']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)


In [20]:
resolved_data['bench'].value_counts()

bench
Sitati, Grace                                                   941
Nyakundi Reuben                                                 897
Mutende, Lilian Nambwire                                        850
 Mshila, Abigail                                                805
Lesootia, Alberty Saitabau                                      785
                                                               ... 
Bwonwong'a, Justus                                                1
Visram, Aleem Alnashir,Kimondo, Kanyi,Ong'udi, Hedwig Imbosa      1
Kimondo, Kanyi,Mutende, Lilian Nambwire                           1
Kendagor, Caroline J.                                             1
Mwera,  Atamba Lynn                                               1
Name: count, Length: 185, dtype: int64

In [21]:
#generate mean time to resolve for each bench by case nature
bench_mean_time = resolved_data.groupby(['bench', 'case_nature'])['time_to_resolve'].mean().reset_index()
bench_mean_time['time_to_resolve'] = bench_mean_time['time_to_resolve'].astype(int)
bench_mean_time = bench_mean_time.sort_values(by=['time_to_resolve'], ascending=False)
bench_mean_time



,bench,case_nature,time_to_resolve
98,"Kendagor, Caroline J.",CIVIL,5966
92,"Kathina, Margaret Kyalo",CIVIL,4288
13,"Mati, Martin Muya",CIVIL,4184
261,"Sitati, Grace",CIVIL,3684
48,"Chirchir,sophie Chebet",CIVIL,3533
...,...,...,...
101,"Kerubo, Orora Rita",CRIMINAL,0
70,"Imaana, Fredrick Koome",CRIMINAL,0
135,"Makau, Nancy Mwende Nzau",CRIMINAL,0
89,"Karanja, Virginia",CRIMINAL,0


In [22]:
'''def merit_resolution(resolved_data):
    merit = []
    other = []
    for i in resolved_data['outcome']:
        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:
            merit.append(1)
            other.append(0)
        else:
            merit.append(0)
            other.append(1)
    resolved_data['merit_resolution'] = merit
    resolved_data['other_resolution'] = other
    return resolved_data

resolved_data = merit_resolution(resolved_data)'''


'def merit_resolution(resolved_data):\n    merit = []\n    other = []\n    for i in resolved_data[\'outcome\']:\n        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:\n            merit.append(1)\n            other.append(0)\n        else:\n            merit.append(0)\n            other.append(1)\n    resolved_data[\'merit_resolution\'] = merit\n    resolved_data[\'other_resolution\'] = other\n    return resolved_data\n\nresolved_data = merit_resolution(resolved_data)'

In [23]:
def merit_resolution(resolved_data):
    resolution_nature = []
    for i in resolved_data['outcome']:
        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:
            resolution_nature.append('merit')
        else:
            resolution_nature.append('other')
    resolved_data['resolution_nature'] = resolution_nature
    return resolved_data

resolved_data = merit_resolution(resolved_data)

In [24]:
resolved_data['resolution_nature'].value_counts()

resolution_nature
other    19542
merit    15540
Name: count, dtype: int64

In [25]:
#function to create a column for resolution case nature using resolution nature and case nature
def resolution_case_nature(resolved_data):
    resolution_case_nature = []
    for i, row in resolved_data.iterrows():
        if row['resolution_nature'] == 'merit' and row['case_nature'] == 'CRIMINAL':
            resolution_case_nature.append('merit_criminal')
        elif row['resolution_nature'] == 'merit' and row['case_nature'] == 'CIVIL':
            resolution_case_nature.append('merit_civil')
        elif row['resolution_nature'] == 'other' and row['case_nature'] == 'CRIMINAL':
            resolution_case_nature.append('other_criminal')
        else:
            row['resolution_nature'] == 'other' and row['case_nature'] == 'CIVIL'
            resolution_case_nature.append('other_civil')
    resolved_data['resolution_case_nature'] = resolution_case_nature
    return resolved_data

resolved_data = resolution_case_nature(resolved_data)

resolved_data['resolution_case_nature'].value_counts()

resolution_case_nature
other_civil       16901
merit_civil        9361
merit_criminal     6179
other_criminal     2641
Name: count, dtype: int64

In [26]:
#resolution case nature by bench
resolution_bench = resolved_data.groupby(['bench', 'resolution_case_nature'])['case_identifier'].count().reset_index()
resolution_bench = resolution_bench.sort_values(by=['case_identifier'], ascending=False)
resolution_bench

#pivot table for resolution case nature by bench
resolution_bench_pivot = resolved_data.pivot_table(index='bench', columns='resolution_case_nature', values='case_identifier', aggfunc='count').reset_index()
resolution_bench_pivot = resolution_bench_pivot.fillna(0)
resolution_bench_pivot = resolution_bench_pivot.sort_values(by=['merit_civil', 'merit_criminal', 'other_civil', 'other_criminal'], ascending=False)
resolution_bench_pivot

#change to integer
resolution_bench_pivot['merit_civil'] = resolution_bench_pivot['merit_civil'].astype(int)
resolution_bench_pivot['merit_criminal'] = resolution_bench_pivot['merit_criminal'].astype(int)
resolution_bench_pivot['other_civil'] = resolution_bench_pivot['other_civil'].astype(int)
resolution_bench_pivot['other_criminal'] = resolution_bench_pivot['other_criminal'].astype(int)
resolution_bench_pivot

#rename columns
resolution_bench_pivot.columns = ['Bench', 'Merit Civil', 'Merit Criminal', 'Others Civil', 'Other Criminal']
resolution_bench_pivot

,Bench,Merit Civil,Merit Criminal,Others Civil,Other Criminal
12,"Aburili, Roselyne",391,160,51,5
131,"Odera, Teresia Achieng",330,182,80,7
80,"Magare, Dennis Kizito",280,27,305,22
124,Nyakundi Reuben,266,277,271,83
93,"Mohochi, Samuel Mukira",241,78,138,15
...,...,...,...,...,...
149,"Opondo, Ase Meresia",0,0,1,0
126,"Nyangena, Theresa",0,0,0,2
144,"Omono, Tobias Odongo",0,0,0,2
67,"Kimondo, Kanyi,Mutende, Lilian Nambwire",0,0,0,1


In [27]:
#generate mean time to resolve for each bench by resolution case nature
bench_mean_time_resolution = resolved_data.groupby(['bench', 'resolution_case_nature'])['time_to_resolve'].mean().reset_index()
bench_mean_time_resolution['time_to_resolve'] = bench_mean_time_resolution['time_to_resolve'].astype(int)
bench_mean_time_resolution = bench_mean_time_resolution.sort_values(by=['time_to_resolve'], ascending=False)
bench_mean_time_resolution

#pivot table for mean time to resolve by bench and resolution case nature
bench_mean_time_resolution_pivot = resolved_data.pivot_table(index='bench', columns='resolution_case_nature', values='time_to_resolve', aggfunc='mean').reset_index()
bench_mean_time_resolution_pivot = bench_mean_time_resolution_pivot.fillna(0)
bench_mean_time_resolution_pivot = bench_mean_time_resolution_pivot.sort_values(by=['merit_civil', 'merit_criminal', 'other_civil', 'other_criminal'], ascending=False)
bench_mean_time_resolution_pivot

#change to integer
bench_mean_time_resolution_pivot['merit_civil'] = bench_mean_time_resolution_pivot['merit_civil'].astype(int)
bench_mean_time_resolution_pivot['merit_criminal'] = bench_mean_time_resolution_pivot['merit_criminal'].astype(int)
bench_mean_time_resolution_pivot['other_civil'] = bench_mean_time_resolution_pivot['other_civil'].astype(int)
bench_mean_time_resolution_pivot['other_criminal'] = bench_mean_time_resolution_pivot['other_criminal'].astype(int)
bench_mean_time_resolution_pivot

#rename columns
bench_mean_time_resolution_pivot.columns = ['Bench', 'Merit Civil Timelines', 'Merit Criminal Timelines', 'Others Civil Timelines', 'Other Criminal Timelines']
bench_mean_time_resolution_pivot

,Bench,Merit Civil Timelines,Merit Criminal Timelines,Others Civil Timelines,Other Criminal Timelines
54,"Kathina, Margaret Kyalo",4288,0,0,0
137,"Odunga,george Vincent",3199,1830,2005,0
28,"Chirchir,sophie Chebet",2865,570,3837,1032
65,"Kimaru, Luka Kiprotich",2795,2102,0,0
147,"Onkoba, Rosemary",2705,751,3241,1282
...,...,...,...,...,...
182,"Waweru, Hatari Peter",0,0,0,875
126,"Nyangena, Theresa",0,0,0,655
67,"Kimondo, Kanyi,Mutende, Lilian Nambwire",0,0,0,51
144,"Omono, Tobias Odongo",0,0,0,5


In [28]:
#merge bench resolution time and bench resolution
bench_performance = pd.merge(resolution_bench_pivot, bench_mean_time_resolution_pivot, on='Bench', how='inner')
bench_performance

#save the combined file as a csv
bench_performance.to_csv(f'{path}/raw data/bench_performance.csv', index=False)